# 04 - Price Prediction (Hero: Price Timing)

Goal: train a baseline model to predict ADR and simulate 'wait N days' to produce a booking decision.

In [ ]:
import faulthandler
import time
from pathlib import Path

# Notebook-friendly faulthandler: avoid UnsupportedOperation: fileno by logging to a file if needed
_debug_log = Path('..') / 'debug_price_prediction.log'
_fh_file = None

try:
    import sys

    _fh_file = getattr(sys, '__stderr__', sys.stderr)
    faulthandler.enable(file=_fh_file)
except Exception:
    _fh_file = open(_debug_log, 'w', encoding='utf-8')
    faulthandler.enable(file=_fh_file)

try:
    faulthandler.dump_traceback_later(15, repeat=True, file=_fh_file)
except TypeError:
    faulthandler.dump_traceback_later(15, repeat=True)

print('Debug enabled: stack traces every 15s while running.', flush=True)
print(f'If traces do not show in the notebook, open: {_debug_log}', flush=True)


def _timed(label: str):
    t0 = time.time()
    print(label, flush=True)

    def _done(extra: str = ''):
        dt = time.time() - t0
        msg = f'{label} done in {dt:.2f}s' + (f' | {extra}' if extra else '')
        print(msg, flush=True)

    return _done


In [ ]:
from pathlib import Path
import sys

sys.path.append(str(Path('..').resolve()))

import pandas as pd
from src.price_model import train_price_model, price_timing_signal, DEFAULT_FEATURE_COLS

_done = _timed('STEP 1: read hotel_bookings.csv')
df = pd.read_csv('../data/hotel_bookings.csv')
_done(extra=f"shape={df.shape}")

_done = _timed('STEP 2: train_price_model (one-hot + HGBR fit)')
pipe, mae = train_price_model(df, feature_cols=DEFAULT_FEATURE_COLS)
_done(extra=f"mae={mae:.4f}")

mae

In [ ]:
_done = _timed('STEP 3: sample + price_timing_signal (wait_days=7)')
sample = df[DEFAULT_FEATURE_COLS].sample(1, random_state=42).iloc[0].to_dict()
result = price_timing_signal(pipe, sample, wait_days=7)
_done(extra=f"decision={result.decision} | pct_change={result.pct_change:.2f}%")

result